<a href="https://colab.research.google.com/github/neshkatrapati/argile/blob/master/MLMPlain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-i64l0ac_
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-i64l0ac_
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 4.1 MB/s 
     |████████████████████████████████| 596 kB 51.1 MB/s 
     |████████████████████████████████| 59 kB 7.0 MB/s 
     |████████████████████████████████| 3.3 MB 36.4 MB/s 
  Created wheel for transformers: filename=transformers-4.13.0.dev0-py3-none-any.whl size=3122766 sha256=529693e2ddadec01752539b18e96b5d988ac387c3c9e4a63ad59e1eb86c5e067
  Stored in directory: /tmp/pip-ephem-wheel-cache-s9ws95b6/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attemp

In [3]:
! wget http://124.123.37.168/ukwac.txt

--2021-11-10 10:00:37--  http://124.123.37.168/ukwac.txt
Connecting to 124.123.37.168:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1662286035 (1.5G) [text/plain]
Saving to: ‘ukwac.txt’

ukwac.txt           100%[===================>]   1.55G  9.33MB/s    in 2m 37s  

2021-11-10 10:03:14 (10.1 MB/s) - ‘ukwac.txt’ saved [1662286035/1662286035]



In [23]:
! du -h ukwac.txt

1.6G	ukwac.txt


In [1]:
! head -n 100000 ukwac.txt > u10L.txt

In [1]:
%%time 
from pathlib import Path

from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

unk_token = "<UNK>"
spl_tokens = ["<UNK>", "<SEP>", "<MASK>", "<CLS>"]  # special tokens

tokenizer = Tokenizer(WordLevel(unk_token = unk_token))
trainer = WordLevelTrainer(vocab_size = 1000000, special_tokens = spl_tokens, min_frequency=25, show_progress=True)
tokenizer.pre_tokenizer = Whitespace()
# # Customize training

#tokenizer.train(['u10L.txt'],trainer)

CPU times: user 5.68 ms, sys: 3.92 ms, total: 9.6 ms
Wall time: 9.61 ms


In [2]:
#tokenizer.save('ukwac-tokenizer.json')
tokenizer = tokenizer.from_file('ukwac-tokenizer.json')

In [3]:
tokenizer.get_vocab_size()

8270

In [4]:
def tokenize(sentence, tokzer):
    
    toks = tokzer.encode(sentence).ids
    #toks = [t for t in toks if t not in REM_TOKS]
    
    return toks

In [5]:
tokenize("the cat sat on the mat", tokenizer)

[5, 4749, 3842, 21, 5, 0]

In [6]:
from tokenizers.processors import BertProcessing
# tokenizer._tokenizer.post_processor = BertProcessing(
#     ("</s>", tokenizer.token_to_id("</s>")),
#     ("<s>", tokenizer.token_to_id("<s>")),
# )
tokenizer.enable_truncation(max_length=100)

In [7]:
!nvidia-smi

Wed Nov 10 14:57:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
import torch
torch.cuda.is_available()

True

In [9]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=int(tokenizer.get_vocab_size()),
    max_position_embeddings=100,
    num_attention_heads=8,
    num_hidden_layers=4,
    type_vocab_size=1,
)

In [10]:
from transformers import PreTrainedTokenizerFast

albert_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)

In [11]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [12]:
model.num_parameters()

35382350

In [13]:
albert_tokenizer('hello world', add_special_tokens=True,
                                        truncation=True, max_length=100)

{'input_ids': [0, 171], 'token_type_ids': [0, 0], 'attention_mask': [1, 1]}

In [14]:
albert_tokenizer.add_special_tokens({'cls_token' : '<CLS>', 'mask_token':'<MASK>', 'unk_token' : '<UNK>', 'sep_token' : '<SEP>', 'pad_token' : '<PAD>' })

1

In [15]:
albert_tokenizer.pad_token_id 

8270

In [16]:

import datasets
from datasets import load_dataset


In [17]:
dset = load_dataset("text", data_files = "u10L.txt")


Using custom data configuration default-35f89ffa5f4233b9
Reusing dataset text (/root/.cache/huggingface/datasets/text/default-35f89ffa5f4233b9/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)


  0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
dataset = dset['train']

In [20]:
dataset = dataset.map(lambda ex: albert_tokenizer(ex["text"], add_special_tokens=True,
                                        truncation=True, max_length=100), num_proc = 4, batched=True)
dataset.set_format(type='torch', columns=['input_ids'])

In [21]:
dataset.save_to_disk('ukwac.dset')


In [18]:
dataset = datasets.load_from_disk('ukwac.dset')

In [19]:
dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'text', 'token_type_ids'],
    num_rows: 100000
})

In [20]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=albert_tokenizer, mlm=True, mlm_probability=0.15
)

In [21]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [22]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./ukwac",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=6,
    save_steps=10000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [23]:
%%time
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: text.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 100000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 16667
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


RuntimeError: ignored

In [28]:
torch.cuda.empty_cache()